In [8]:
# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import time
import os
import math

import numpy as np
from numpy.random import seed
from numpy.random import rand

import pybullet as p
import pybullet_data

# seed random number generator
seed(1)

In [9]:
# now we connect to GUI
physicsClient = p.connect(p.GUI,options="--opengl2")
# physicsClient = p.connect(p.DIRECT)   

# gravity
p.setGravity(0,0,-10)

# set urdf path 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setRealTimeSimulation(0)

#load urdf
planeId = p.loadURDF("plane.urdf")

argv[0]=--opengl2
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = --opengl2
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
4

	visual 0x21 selected
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [10]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
shovelId = p.loadURDF('./urdf/shovel/shovelFlat.urdf', startPos, startOrientation)

# get shovel position 
shovel = p.getBasePositionAndOrientation(shovelId)
    
# simulate
for i in range (500):
    p.stepSimulation()
    time.sleep(0.001)


In [11]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(shovelId, 0)

# admissible init positions
minpos = np.asarray(shovelLinkState[0]) - 3*1e-1
maxpos = np.asarray(shovelLinkState[0]) + 3*1e-1

# number of pebbles
pebbleNum = 50

# store pebbles IDs
pebbleId = np.zeros(pebbleNum)

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    pebbleId[i] = p.loadURDF('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)
    
# simulate
for i in range (500):
    p.stepSimulation()
    time.sleep(0.001)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: base_link
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inerti

In [12]:
# test move with pos

# max force on joint
maxForce = 1000

# target pos and vel - prismatic
targetPosMean = 0
targetPosStd = 3e-1
targetVelPos = 0.5

# target pos and vel - revolute
targetAngMean = 0
targetAngStd = 5e-1
targetVelAng = 1

# current control mode
mode = p.VELOCITY_CONTROL

# control
for i in range(5):
    
    # redefine target Pos and Vel    
    targetPos= i%2*(targetPosMean+targetPosStd)+(1-i%2)*(targetPosMean-targetPosStd)
    targetAng= i%2*(targetAngMean+targetAngStd)+(1-i%2)*(targetAngMean-targetAngStd)   
    
    # error init - Pos
    shovelJointPos = p.getJointState(shovelId,0)
    ePos = targetPos-shovelJointPos[0]
    
    # error init - Ang
    # shovelJointAng = p.getJointState(shovelId,1)
    # eAng = targetAng-shovelJointAng[0]
    
    # control
    while np.abs(ePos) > 1e-2:                                    
        
        # control - Pos
        targetVelPosCtrl = targetVelPos*np.sign(ePos)
        shovelContr = p.setJointMotorControl2(bodyIndex=shovelId, jointIndex=0, controlMode=mode, targetVelocity=targetVelPosCtrl, force=maxForce)    
        
        # control - Ang
        # targetVelAngCtrl = targetVelAng*np.sign(eAng)
        # shovelContr = p.setJointMotorControl2(bodyIndex=shovelId, jointIndex=1, controlMode=mode, targetVelocity=targetVelAngCtrl, force=maxForce)    
        
        # info on target
        # print(str(targetPos), str(shovelPos[0]), str(targetVelCtrl))            
        
        # step
        p.stepSimulation()
        
        # sleep
        time.sleep(1/240)
        
        # error update - Pos
        shovelJointPos = p.getJointState(shovelId,0)
        ePos = targetPos-shovelJointPos[0] 
        
        # error update - Ang
        # shovelJointAng = p.getJointState(shovelId,1)
        # eAng = targetAng-shovelJointAng[0]           

In [13]:
# disconnect engine
time.sleep(2)
p.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
